# Binance Balance

In [ ]:
from clients import BinanceClient

binance = BinanceClient()
binance.fetch_balance()

symbol = 'BTC/USDT'
timeframe = '1h'
df = binance.get_ohlc(symbol, timeframe)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Candlestick(x=df['datetime'],
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'])])

fig.show()

# Ethereum Mempool

In [1]:
from clients import EthClient
ec = EthClient()

True


In [7]:
import json
import time

while True:
    func_obj, results = ec.get_pending_tx_uniswap()
    if results != []:
        break
    time.sleep(10)
        # output json
with open('./data/uniswap_pending.json', 'w') as f:
    json.dump(results, f)

In [3]:
results

[{'params': {'tokenIn': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2',
   'tokenOut': '0x77E06c9eCCf2E797fd462A92B6D7642EF85b0A44',
   'fee': 10000,
   'recipient': '0x90C9BceCd185b9a70ea1CCaea0cFe9e1Fb007335',
   'deadline': 1712246687,
   'amountIn': 48809677473410079,
   'amountOutMinimum': 263641344,
   'sqrtPriceLimitX96': 0},
  'timestamp': '2024-04-04T11:49:58.397172'}]